# PEFT 库 LoRA 实战 - OpenAI Whisper-large-v2


In [2]:
model_name_or_path = "openai/whisper-large-v2"
model_dir = "../model/whisper-large-v2-asr-int8"

language = "Chinese (China)"
language_abbr = "zh-CN"
task = "transcribe"
dataset_name = "mozilla-foundation/common_voice_11_0"

batch_size=64
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

# 下载数据集 Common Voice

In [3]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()


In [3]:
common_voice["train"] = load_dataset(dataset_name, language_abbr, split="train", trust_remote_code=True,cache_dir="../data")
common_voice["validation"] = load_dataset(dataset_name, language_abbr, split="validation", trust_remote_code=True,cache_dir="../data")

In [4]:
! pip install soundfile
! pip install librosa

In [5]:
common_voice["train"][0]


{'client_id': '95368aab163e0387e4fd4991b4f2d8ccfbd4364bf656c860230501fd27dcedf087773e4695a6cf5de9c4f1d406d582283190d065cdfa36b0e2b060cffaca977e',
 'path': '/home/jovyan/work/data/downloads/extracted/165d082818f84509a4b33d6bb878b527c029489a2976709c928a3ce219ce5b47/zh-CN_train_0/common_voice_zh-CN_33211332.mp3',
 'audio': {'path': '/home/jovyan/work/data/downloads/extracted/165d082818f84509a4b33d6bb878b527c029489a2976709c928a3ce219ce5b47/zh-CN_train_0/common_voice_zh-CN_33211332.mp3',
  'array': array([-6.82121026e-13, -2.27373675e-12, -2.27373675e-12, ...,
          1.21667399e-05,  3.23003678e-06, -2.43066324e-07]),
  'sampling_rate': 48000},
 'sentence': '性喜温暖润湿气候且耐寒。',
 'up_votes': 2,
 'down_votes': 0,
 'age': '',
 'gender': '',
 'accent': '',
 'locale': 'zh-CN',
 'segment': ''}

# 预处理训练数据集


In [6]:
from transformers import AutoFeatureExtractor, AutoTokenizer, AutoProcessor

# 从预训练模型加载特征提取器
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name_or_path)

# 从预训练模型加载分词器，可以指定语言和任务以获得最适合特定需求的分词器配置
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, language=language, task=task)

# 从预训练模型加载处理器，处理器通常结合了特征提取器和分词器，为特定任务提供一站式的数据预处理
processor = AutoProcessor.from_pretrained(model_name_or_path, language=language, task=task)

# 移除数据集中不必要的字段


In [7]:
common_voice = common_voice.remove_columns(
    ["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"]
)

In [8]:
common_voice["train"][0]

{'audio': {'path': '/home/jovyan/work/data/downloads/extracted/165d082818f84509a4b33d6bb878b527c029489a2976709c928a3ce219ce5b47/zh-CN_train_0/common_voice_zh-CN_33211332.mp3',
  'array': array([-6.82121026e-13, -2.27373675e-12, -2.27373675e-12, ...,
          1.21667399e-05,  3.23003678e-06, -2.43066324e-07]),
  'sampling_rate': 48000},
 'sentence': '性喜温暖润湿气候且耐寒。'}

# 降采样音频数据

In [9]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [10]:
# sampling_rate 从 48KHZ 降为 16KHZ
common_voice["train"][0]

{'audio': {'path': '/home/jovyan/work/data/downloads/extracted/165d082818f84509a4b33d6bb878b527c029489a2976709c928a3ce219ce5b47/zh-CN_train_0/common_voice_zh-CN_33211332.mp3',
  'array': array([ 5.09317033e-11, -7.27595761e-12, -6.54836185e-11, ...,
         -5.96661994e-06,  2.71382887e-05,  1.29687978e-05]),
  'sampling_rate': 16000},
 'sentence': '性喜温暖润湿气候且耐寒。'}

In [11]:
def prepare_dataset(batch):
    audio = batch["audio"]
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

# 数据抽样

In [12]:
small_common_voice = DatasetDict()

small_common_voice["train"] = common_voice["train"].shuffle(seed=16).select(range(640))
small_common_voice["validation"] = common_voice["validation"].shuffle(seed=16).select(range(320))

In [13]:
small_common_voice

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 640
    })
    validation: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 320
    })
})

# 如果全量训练，则使用完整数据代替抽样

In [14]:
# 抽样数据处理
tokenized_common_voice = small_common_voice.map(prepare_dataset)

# 完整数据训练，尝试开启 `num_proc=8` 参数多进程并行处理（如阻塞无法运行，则不使用此参数）
# tokenized_common_voice = common_voice.map(prepare_dataset, num_proc=8)

In [15]:
tokenized_common_voice


DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence', 'input_features', 'labels'],
        num_rows: 640
    })
    validation: Dataset({
        features: ['audio', 'sentence', 'input_features', 'labels'],
        num_rows: 320
    })
})

# 自定义语音数据整理器

In [16]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

In [17]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

# 定义一个针对语音到文本任务的数据整理器类
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any  # 处理器结合了特征提取器和分词器

    # 整理器函数，将特征列表处理成一个批次
    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # 从特征列表中提取输入特征，并填充以使它们具有相同的形状
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # 从特征列表中提取标签特征（文本令牌），并进行填充
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # 使用-100替换标签中的填充区域，-100通常用于在损失计算中忽略填充令牌
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # 如果批次中的所有序列都以句子开始令牌开头，则移除它
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        # 将处理过的标签添加到批次中
        batch["labels"] = labels

        return batch  # 返回最终的批次，准备好进行训练或评估

In [18]:
# 用给定的处理器实例化数据整理器
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

# 模型准备

In [19]:
from transformers import AutoModelForSpeechSeq2Seq

model = AutoModelForSpeechSeq2Seq.from_pretrained (model_name_or_path, load_in_8bit=True, device_map="auto")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [20]:
! pip install -U bitsandbytes
!pip install peft

In [21]:
# 设置模型配置中的forced_decoder_ids属性为None
model.config.forced_decoder_ids = None  # 这通常用于指定在解码（生成文本）过程中必须使用的特定token的ID，设置为None表示没有这样的强制要求

# 设置模型配置中的suppress_tokens列表为空
model.config.suppress_tokens = []  # 这用于指定在生成过程中应被抑制（不生成）的token的列表，设置为空列表表示没有要抑制的token

# PEFT 微调前的模型处理

In [22]:
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

# LoRA Adapter 配置
在 peft 中使用LoRA非常简捷，借助 PeftModel抽象，我们可以快速使用低秩适配器（LoRA）到任意模型。

通过使用 peft 中的 get_peft_model 工具函数来实现。

关于 LoRA 超参数的说明：
MatMul(B,A) * Scaling
Scaling = LoRA_Alpha / Rank

In [23]:
from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

# 创建一个LoraConfig对象，用于设置LoRA（Low-Rank Adaptation）的配置参数
config = LoraConfig(
    r=4,  # LoRA的秩，影响LoRA矩阵的大小
    lora_alpha=64,  # LoRA适应的比例因子
    # 指定将LoRA应用到的模型模块，通常是attention和全连接层的投影。
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,  # 在LoRA模块中使用的dropout率
    bias="none",  # 设置bias的使用方式，这里没有使用bias
)

# 使用get_peft_model函数和给定的配置来获取一个PEFT模型


In [24]:
peft_model = get_peft_model(model, config)


In [25]:
# 打印 LoRA 微调训练的模型参数
peft_model.print_trainable_parameters()

trainable params: 1,966,080 || all params: 1,545,271,040 || trainable%: 0.1272


# 模型训练
Seq2SeqTrainingArguments 训练参数
关于设置训练步数和评估步数

基于 epochs 设置：

    num_train_epochs=3,  # 训练的总轮数
    evaluation_strategy="epoch",  # 设置评估策略，这里是在每个epoch结束时进行评估
    warmup_steps=50,  # 在训练初期增加学习率的步数，有助于稳定训练
基于 steps 设置：

    max_steps=100, # 训练总步数
    evaluation_strategy="steps", 
    eval_steps=25, # 评估步数

In [26]:
from transformers import Seq2SeqTrainingArguments

# 设置序列到序列模型训练的参数
training_args = Seq2SeqTrainingArguments(
    output_dir=model_dir,  # 指定模型输出和保存的目录
    per_device_train_batch_size=batch_size,  # 每个设备上的训练批量大小
    learning_rate=1e-3,  # 学习率
    num_train_epochs=1,  # 训练的总轮数
    eval_strategy="epoch",  # 设置评估策略，这里是在每个epoch结束时进行评估
    # warmup_steps=50,  # 在训练初期增加学习率的步数，有助于稳定训练
    # fp16=True,  # 启用混合精度训练，可以提高训练速度，同时减少内存使用
    per_device_eval_batch_size=batch_size,  # 每个设备上的评估批量大小
    generation_max_length=128,  # 生成任务的最大长度
    logging_steps=10,  # 指定日志记录的步骤，用于跟踪训练进度
    remove_unused_columns=False,  # 是否删除不使用的列，以减少数据处理开销
    label_names=["labels"],  # 指定标签列的名称，用于训练过程中
    # evaluation_strategy="steps",
    # eval_steps=25,
)

# 实例化 Seq2SeqTrainer 训练器


In [27]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=peft_model,
    train_dataset=tokenized_common_voice["train"],
    eval_dataset=tokenized_common_voice["validation"],
    data_collator=data_collator,
    tokenizer=processor.feature_extractor,
)
peft_model.config.use_cache = False

/tmp/ipykernel_1859049/3403953682.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [28]:
trainer.train()

/opt/conda/lib/python3.11/site-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/python3.11/site-packages/torch/utils/checkpoint.py:86: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Epoch,Training Loss,Validation Loss
1,1.499000,1.077402


/opt/conda/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


TrainOutput(global_step=10, training_loss=1.499030876159668, metrics={'train_runtime': 173.7431, 'train_samples_per_second': 3.684, 'train_steps_per_second': 0.058, 'total_flos': 1.36064139264e+18, 'train_loss': 1.499030876159668, 'epoch': 1.0})

# 保存 LoRA 模型(Adapter)

In [29]:
trainer.save_model(model_dir)


In [30]:
peft_model.eval()


PeftModel(
  (base_model): LoraModel(
    (model): WhisperForConditionalGeneration(
      (model): WhisperModel(
        (encoder): WhisperEncoder(
          (conv1): Conv1d(80, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
          (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
          (embed_positions): Embedding(1500, 1280)
          (layers): ModuleList(
            (0-31): 32 x WhisperEncoderLayer(
              (self_attn): WhisperAttention(
                (k_proj): Linear8bitLt(in_features=1280, out_features=1280, bias=False)
                (v_proj): lora.Linear8bitLt(
                  (base_layer): Linear8bitLt(in_features=1280, out_features=1280, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=1280, out_features=4, bias=False)
                  )
                

# 模型推理

In [9]:
model_dir = "../model/whisper-large-v2-asr-int8"

language = "Chinese (China)"
language_abbr = "zh-CN"
language_decode = "chinese"
task = "transcribe"

In [10]:
from transformers import AutoModelForSpeechSeq2Seq, AutoTokenizer, AutoProcessor
from peft import PeftConfig, PeftModel

peft_config = PeftConfig.from_pretrained(model_dir)

base_model = AutoModelForSpeechSeq2Seq.from_pretrained(
    peft_config.base_model_name_or_path, load_in_8bit=True, device_map="cuda"
)

peft_model = PeftModel.from_pretrained(base_model, model_dir)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [11]:
tokenizer = AutoTokenizer.from_pretrained(peft_config.base_model_name_or_path, language=language, task=task)
processor = AutoProcessor.from_pretrained(peft_config.base_model_name_or_path, language=language, task=task)
feature_extractor = processor.feature_extractor

In [34]:
test_audio = "../data/audio/test_zh.flac"


In [35]:
from transformers import AutomaticSpeechRecognitionPipeline

pipeline = AutomaticSpeechRecognitionPipeline(model=peft_model, tokenizer=tokenizer, feature_extractor=feature_extractor)

forced_decoder_ids = processor.get_decoder_prompt_ids(language=language_decode, task=task)

Device set to use cuda


In [36]:
import torch

with torch.cuda.amp.autocast():
    text = pipeline(test_audio)["text"]

/tmp/ipykernel_1859049/232075547.py:3: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
/opt/conda/lib/python3.11/site-packages/transformers/models/whisper/generation_whisper.py:604: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.
/opt/conda/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:185:

In [37]:
text

'这是一段测试用于Whisper、Large、V2模型的自动语音识别测试。'

# OpenAI Whisper LoRA 模型评估和推理

## 评估数据集处理

In [38]:
from datasets import load_dataset, DatasetDict, Audio

common_voice = DatasetDict()
common_voice["test"] = load_dataset(dataset_name, language_abbr, split="test", trust_remote_code=True)
common_voice = common_voice.remove_columns(
    ["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"]
)
common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [7]:
def prepare_dataset(batch):
    audio = batch["audio"]
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [40]:
small_common_voice = DatasetDict()

small_common_voice["test"] = common_voice["test"].shuffle(seed=16).select(range(320))

In [41]:
tokenized_common_voice = small_common_voice.map(prepare_dataset)


In [13]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

# 定义一个针对语音到文本任务的数据整理器类
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any  # 处理器结合了特征提取器和分词器

    # 整理器函数，将特征列表处理成一个批次
    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # 从特征列表中提取输入特征，并填充以使它们具有相同的形状
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # 从特征列表中提取标签特征（文本令牌），并进行填充
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # 使用-100替换标签中的填充区域，-100通常用于在损失计算中忽略填充令牌
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # 如果批次中的所有序列都以句子开始令牌开头，则移除它
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        # 将处理过的标签添加到批次中
        batch["labels"] = labels

        return batch  # 返回最终的批次，准备好进行训练或评估

# 用给定的处理器实例化数据整理器
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

# 评估模型

In [43]:
import evaluate

# 词错误率（WER）是评估ASR模型常用的指标。从 Evaluate 加载 WER 指标
metric = evaluate.load("wer")

In [44]:
!pip install jiwer

In [45]:
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import gc

eval_dataloader = DataLoader(tokenized_common_voice["test"], batch_size=batch_size, collate_fn=data_collator)

In [46]:
# 遍历评估数据加载器中的所有批次
for step, batch in enumerate(tqdm(eval_dataloader)):
    # 使用自动混合精度来加速计算，并减少显存使用
    with torch.cuda.amp.autocast():
        # 不计算梯度，以节省计算资源，仅用于推理和评估
        with torch.no_grad():
            # 生成预测的标记(tokens)，这里使用模型的generate函数进行文本生成
            generated_tokens = (
                peft_model.generate(
                    input_features=batch["input_features"].to("cuda"),  # 将输入特征移动到GPU上
                    decoder_input_ids=batch["labels"][:, :4].to("cuda"),  # 提供解码器的初始输入
                    max_new_tokens=255,  # 设置生成的最大新标记数量
                )
                .cpu()  # 将生成的标记移回CPU
                .numpy()  # 转换为NumPy数组以便进一步处理
            )
            # 获取批次中的标签，并将其移回CPU
            labels = batch["labels"].cpu().numpy()
            # 将标签中的-100替换为填充标记的ID，-100通常用于忽略计算损失的标记
            labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
            # 使用分词器解码生成的标记和标签，以获得可读的文本
            decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
            # 将预测和参考添加到评估指标中，用于后续的性能评估
            metric.add_batch(
                predictions=decoded_preds,
                references=decoded_labels,
            )
    # 删除不再需要的变量以释放内存
    del generated_tokens, labels, batch
    # 手动触发垃圾收集，进一步清理内存
    gc.collect()

  0%|          | 0/5 [00:00<?, ?it/s]/tmp/ipykernel_1859049/2829552596.py:4: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
100%|██████████| 5/5 [01:29<00:00, 17.97s/it]


In [47]:
# 计算词错误率（WER）指标，并将结果转换为百分比形式
wer = 100 * metric.compute()

# 打印词错误率，f"{wer=}"是一种格式化字符串的简洁写法，它会展示变量名和值
print(f"{wer=}%")

wer=100.0%


# 使用全量调参数

In [71]:
common_voice = DatasetDict()
common_voice["train"] = load_dataset(dataset_name, language_abbr, split="train", trust_remote_code=True,cache_dir="../data")
common_voice["validation"] = load_dataset(dataset_name, language_abbr, split="validation", trust_remote_code=True,cache_dir="../data")
# 采样率 -》16000
common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))
# 删除不需要的字段
common_voice = common_voice.remove_columns(
    ["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"]
)

common_voice["train"] = common_voice["train"].shuffle(seed=16)
common_voice["validation"] = common_voice["validation"].shuffle(seed=16)

In [72]:
tokenized_common_voice = common_voice.map(prepare_dataset)

In [73]:
model = AutoModelForSpeechSeq2Seq.from_pretrained (model_name_or_path, load_in_8bit=True, device_map="auto")


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [74]:

# 设置模型配置中的forced_decoder_ids属性为None
model.config.forced_decoder_ids = None  # 这通常用于指定在解码（生成文本）过程中必须使用的特定token的ID，设置为None表示没有这样的强制要求

# 设置模型配置中的suppress_tokens列表为空
model.config.suppress_tokens = []  # 这用于指定在生成过程中应被抑制（不生成）的token的列表，设置为空列表表示没有要抑制的token


In [75]:
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

In [76]:
from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

# 创建一个LoraConfig对象，用于设置LoRA（Low-Rank Adaptation）的配置参数
config = LoraConfig(
    r=4,  # LoRA的秩，影响LoRA矩阵的大小
    lora_alpha=64,  # LoRA适应的比例因子
    # 指定将LoRA应用到的模型模块，通常是attention和全连接层的投影。
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,  # 在LoRA模块中使用的dropout率
    bias="none",  # 设置bias的使用方式，这里没有使用bias
)


peft_model = get_peft_model(model, config)

In [77]:
peft_model = get_peft_model(model, config)


# 打印 LoRA 微调训练的模型参数
peft_model.print_trainable_parameters()


trainable params: 1,966,080 || all params: 1,545,271,040 || trainable%: 0.1272


/opt/conda/lib/python3.11/site-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/peft/tuners/tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [78]:
from transformers import Seq2SeqTrainingArguments

# 设置序列到序列模型训练的参数
training_args = Seq2SeqTrainingArguments(
    output_dir=model_dir,  # 指定模型输出和保存的目录
    per_device_train_batch_size=batch_size,  # 每个设备上的训练批量大小
    learning_rate=1e-3,  # 学习率
    num_train_epochs=3,  # 训练的总轮数
    eval_strategy="epoch",  # 设置评估策略，这里是在每个epoch结束时进行评估
    # warmup_steps=50,  # 在训练初期增加学习率的步数，有助于稳定训练
    # fp16=True,  # 启用混合精度训练，可以提高训练速度，同时减少内存使用
    per_device_eval_batch_size=batch_size,  # 每个设备上的评估批量大小
    generation_max_length=128,  # 生成任务的最大长度
    logging_steps=10,  # 指定日志记录的步骤，用于跟踪训练进度
    remove_unused_columns=False,  # 是否删除不使用的列，以减少数据处理开销
    label_names=["labels"],  # 指定标签列的名称，用于训练过程中
    # evaluation_strategy="steps",
    # eval_steps=25,
)

In [79]:


from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=peft_model,
    train_dataset=tokenized_common_voice["train"],
    eval_dataset=tokenized_common_voice["validation"],
    data_collator=data_collator,
    tokenizer=processor.feature_extractor,
)
peft_model.config.use_cache = False

/tmp/ipykernel_1859049/3541762594.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [80]:
trainer.train()

/opt/conda/lib/python3.11/site-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/python3.11/site-packages/torch/utils/checkpoint.py:86: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Epoch,Training Loss,Validation Loss
1,0.374400,0.398177
2,0.259800,0.383454
3,0.213500,0.385493


/opt/conda/lib/python3.11/site-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/python3.11/site-packages/torch/utils/checkpoint.py:86: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/opt/conda/lib/python3.11/site-packages/torch/_dynamo/eval_frame.py:838: UserWarni

TrainOutput(global_step=1362, training_loss=0.30237352290079983, metrics={'train_runtime': 21665.5727, 'train_samples_per_second': 4.023, 'train_steps_per_second': 0.063, 'total_flos': 1.85319357677568e+20, 'train_loss': 0.30237352290079983, 'epoch': 3.0})

In [58]:
trainer.save_model(model_dir)

In [82]:
peft_model.eval()

PeftModel(
  (base_model): LoraModel(
    (model): WhisperForConditionalGeneration(
      (model): WhisperModel(
        (encoder): WhisperEncoder(
          (conv1): Conv1d(80, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
          (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
          (embed_positions): Embedding(1500, 1280)
          (layers): ModuleList(
            (0-31): 32 x WhisperEncoderLayer(
              (self_attn): WhisperAttention(
                (k_proj): Linear8bitLt(in_features=1280, out_features=1280, bias=False)
                (v_proj): lora.Linear8bitLt(
                  (base_layer): Linear8bitLt(in_features=1280, out_features=1280, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=1280, out_features=4, bias=False)
                  )
                

# 模型全量数据微调后-评估

In [4]:
from datasets import load_dataset, DatasetDict, Audio

common_voice = DatasetDict()
common_voice["test"] = load_dataset(dataset_name, language_abbr, split="test", trust_remote_code=True)
common_voice = common_voice.remove_columns(
    ["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"]
)
common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [5]:
test_common_voice = DatasetDict()

test_common_voice["test"] = common_voice["test"].shuffle(seed=16)

In [14]:
tokenized_common_voice = test_common_voice.map(prepare_dataset)

from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import gc

eval_dataloader = DataLoader(tokenized_common_voice["test"], batch_size=batch_size, collate_fn=data_collator)

In [15]:
import evaluate
# 词错误率（WER）是评估ASR模型常用的指标。从 Evaluate 加载 WER 指标
metric = evaluate.load("wer")

In [16]:
# 遍历评估数据加载器中的所有批次
for step, batch in enumerate(tqdm(eval_dataloader)):
    # 使用自动混合精度来加速计算，并减少显存使用
    with torch.cuda.amp.autocast():
        # 不计算梯度，以节省计算资源，仅用于推理和评估
        with torch.no_grad():
            # 生成预测的标记(tokens)，这里使用模型的generate函数进行文本生成
            generated_tokens = (
                peft_model.generate(
                    input_features=batch["input_features"].to("cuda"),  # 将输入特征移动到GPU上
                    decoder_input_ids=batch["labels"][:, :4].to("cuda"),  # 提供解码器的初始输入
                    max_new_tokens=255,  # 设置生成的最大新标记数量
                )
                .cpu()  # 将生成的标记移回CPU
                .numpy()  # 转换为NumPy数组以便进一步处理
            )
            # 获取批次中的标签，并将其移回CPU
            labels = batch["labels"].cpu().numpy()
            # 将标签中的-100替换为填充标记的ID，-100通常用于忽略计算损失的标记
            labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
            # 使用分词器解码生成的标记和标签，以获得可读的文本
            decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
            # 将预测和参考添加到评估指标中，用于后续的性能评估
            metric.add_batch(
                predictions=decoded_preds,
                references=decoded_labels,
            )
    # 删除不再需要的变量以释放内存
    del generated_tokens, labels, batch
    # 手动触发垃圾收集，进一步清理内存
    gc.collect()

  0%|          | 0/166 [00:00<?, ?it/s]/tmp/ipykernel_2007215/2829552596.py:4: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.
/opt/conda/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 

In [17]:
# 计算词错误率（WER）指标，并将结果转换为百分比形式
wer = 100 * metric.compute()

# 打印词错误率，f"{wer=}"是一种格式化字符串的简洁写法，它会展示变量名和值
print(f"{wer=}%")

wer=99.97164997164997%


# 总结
## 部分数据调参数 num_train_epochs=1
wer=100.0%
## 全量数据调参数 num_train_epochs=3
wer=99.97164997164997%

## 错误率随有下降 但是效果不明显 训练的轮次太少的原因？